In [4]:
import torch
torch.cuda.is_available()

False

In [5]:
torch.cuda.device_count()

0

In [6]:
torch.cuda.get_device_name(0)

AssertionError: Torch not compiled with CUDA enabled

In [1]:
# Ticket data
import pandas as pd

df = pd.read_excel("../data/JIRA-Export-13.10.2022.xlsx", engine="openpyxl")
df.head();

In [2]:
df = df[["Beschreibung", "Benötigte Zeit"]]
df = df.rename(columns={"Beschreibung":"x", "Benötigte Zeit":"y"})
df["y"] = df["y"].astype(str)
df["x"] = df["x"].astype(str)
df;

In [18]:
# Load Base Model
from transformers import AutoTokenizer

BASE_MODEL = "bert-base-german-cased"
#BASE_MODEL = "bert-base-cased"
LEARNING_RATE = 2e-5
MAX_LENGTH = 256
BATCH_SIZE = 16 # perhaps use 8 to 16
EPOCHS = 20 # 20 -> use stop condition

#id2label = #TODO: Define {0,1,2,.. : 1h,2h,4h,..} 
#label2id = #TODO: inverted id2label

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)


In [5]:
from transformers import AutoModelForSequenceClassification

# classification:
#model = AutoModelForSequenceClassification.from_pretrained(BASE_MODEL, id2label=id2label, label2id=label2id)

# regression:
model = AutoModelForSequenceClassification.from_pretrained(BASE_MODEL, num_labels=1)

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoi

In [6]:
# Tokenize input
def token_preprocessor(df):
    label = df["y"] 
    df = tokenizer(df["x"], padding="max_length", max_length=256, truncation=True)
    # classification:
    # df["label"] = label
    # regression
    df["label"] = float(label)
    #print(df)
    return df
    #return tokenizer(df["x"], padding="max_length", max_length=256, truncation=True)

In [13]:
# Prepare Dataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import Dataset

train, test = train_test_split(df, test_size=0.20, random_state=123)
#X_train, X_test, y_train, y_test = train_test_split(df["X"], df["y"], test_size=0.20, random_state=123)

ds = {"train": Dataset.from_dict(train), "test": Dataset.from_dict(test)}

In [14]:
for split in ds:
    ds[split] = ds[split].map(token_preprocessor, remove_columns=["x", "y"])

  0%|          | 0/3543 [00:00<?, ?ex/s]

  0%|          | 0/886 [00:00<?, ?ex/s]

In [15]:
ds

{'train': Dataset({
     features: ['input_ids', 'token_type_ids', 'attention_mask', 'label'],
     num_rows: 3543
 }),
 'test': Dataset({
     features: ['input_ids', 'token_type_ids', 'attention_mask', 'label'],
     num_rows: 886
 })}

In [16]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

def compute_metrics_for_regression(eval_pred):
    logits, labels = eval_pred
    labels = labels.reshape(-1, 1)
    
    mse = mean_squared_error(labels, logits)
    mae = mean_absolute_error(labels, logits)
    r2 = r2_score(labels, logits)
    single_squared_errors = ((logits - labels).flatten()**2).tolist()
    
    # Compute accuracy 
    # Based on the fact that the rounded score = true score only if |single_squared_errors| < 0.5
    accuracy = sum([1 for e in single_squared_errors if e < 0.25]) / len(single_squared_errors)
    
    return {"mse": mse, "mae": mae, "r2": r2, "accuracy": accuracy}

In [24]:
# Training Params
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="../trainoutput/",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    metric_for_best_model="accuracy",
    load_best_model_at_end=True,
    weight_decay=0.01,
)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
import torch
from transformers import Trainer

class RegressionTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs[0][:, 0]
        loss = torch.nn.functional.mse_loss(logits, labels)
        return (loss, outputs) if return_outputs else loss
    
trainer = RegressionTrainer(
    model=model,
    args=training_args,
    train_dataset=ds["train"],
    eval_dataset=ds["test"],
    compute_metrics=compute_metrics_for_regression
)

trainer.train()

***** Running training *****
  Num examples = 3543
  Num Epochs = 20
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 4440


Epoch,Training Loss,Validation Loss,Mse,Mae,R2,Accuracy
1,No log,4102057472.000000,4102057472.000000,34806.929688,-0.419134,0.000000
2,No log,4101838848.000000,4101838848.000000,34803.785156,-0.419059,0.000000
3,3301905268.736000,4101632768.000000,4101632768.000000,34800.828125,-0.418987,0.000000
4,3301905268.736000,4101438208.000000,4101438208.000000,34798.035156,-0.418920,0.000000
5,3298897166.336000,4101255680.000000,4101255424.000000,34795.410156,-0.418857,0.000000
6,3298897166.336000,4101084928.000000,4101084416.000000,34792.953125,-0.418798,0.000000
7,3171325050.880000,4100925696.000000,4100925440.000000,34790.664062,-0.418743,0.000000
8,3171325050.880000,4100778240.000000,4100777984.000000,34788.542969,-0.418692,0.000000
9,3171325050.880000,4100642560.000000,4100642816.000000,34786.601562,-0.418645,0.000000
10,3294076862.464000,4100519168.000000,4100519168.000000,34784.820312,-0.418602,0.000000


***** Running Evaluation *****
  Num examples = 886
  Batch size = 16
Saving model checkpoint to ../trainoutput/checkpoint-222
Configuration saved in ../trainoutput/checkpoint-222\config.json
Model weights saved in ../trainoutput/checkpoint-222\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 886
  Batch size = 16
Saving model checkpoint to ../trainoutput/checkpoint-444
Configuration saved in ../trainoutput/checkpoint-444\config.json
Model weights saved in ../trainoutput/checkpoint-444\pytorch_model.bin
Deleting older checkpoint [..\trainoutput\checkpoint-886] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 886
  Batch size = 16
Saving model checkpoint to ../trainoutput/checkpoint-666
Configuration saved in ../trainoutput/checkpoint-666\config.json
Model weights saved in ../trainoutput/checkpoint-666\pytorch_model.bin
Deleting older checkpoint [..\trainoutput\checkpoint-444] due to args.save_total_limit
***** Running Evaluation *****
  Num 

In [203]:
from transformers import TextClassificationPipeline

pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True)

In [217]:
test_string1 = "Das Icon muss ausgewechselt werden! Lorem ipsum dolor sit amet, dnmsakd \n hahah IT User, Oberfläche"
test_string2 = """IST: 
Wird im EHB "Stichprobenbildung ohne Ergebniserfassung" im Abschnitt "Stichprobenbildung vollstationär ohne Ergebniserfassung" "Regelprüfung" markiert, dann ist im Abschnitt "Ergänzende Stichprobe bei Anlass- /Wiederholungsprüfung" über den Button "Weiteres" eine weitere Person erfassbar. 

Soll: 
Im Abschnitt "Ergänzende Stichprobe bei Anlass- /Wiederholungsprüfung" soll ausschließlich eine Person erfassbar sein, wenn in Abschnitt "Stichprobenbildung vollstationär ohne Ergebniserfassung" "Anlass/Wiederholungsprüfung" markiert wurde 
"""
test_string3 = "Frage 3.5 (E Frage) wurde im EHB Einrichtung ausgefüllt und \"Empfehlung geben\" angekreuzt \nIm Produktergebnis wird Frage 3.5 nicht in die Tabelle(Empfehlungen zur Beseitigung von Qualitätsdefiziten) aufgenommen"
some_strings = [test_string1, test_string2, test_string3]

In [218]:
encoded = tokenizer(some_strings, padding="max_length", max_length=256, truncation=True, return_tensors="pt")

In [219]:
model(**encoded).logits.reshape(-1).tolist()

[18.531217575073242, 18.532835006713867, 18.53243637084961]